<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# EDA 
<!--- @wandbcode{course-lesson1} -->

In this notebook, we will download a sample of the [BDD100K](https://www.bdd100k.com/) semantic segmentation dataset and use W&B Artifacts and Tables to version and analyze our data. 

In [12]:
DEBUG = False # set this flag to True to use a small subset of data for testing

In [13]:
from fastai.vision.all import *
import src.params as params

import wandb

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


We have defined some global configuration parameters in the `params.py` file. `ENTITY` should correspond to your W&B Team name if you work in a team, replace it with `None` if you work individually. 

In the section below, we will use `untar_data` function from `fastai` to download and unzip our datasets. 

In [14]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [15]:
path = Path(untar_data(URL, force_download=True))

In [16]:
path.ls()

(#3) [Path('/home/mazen/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/home/mazen/.fastai/data/bdd_simple_1k/images'),Path('/home/mazen/.fastai/data/bdd_simple_1k/labels')]

Here we define several functions to help us process the data and upload it as a `Table` to W&B. 

In [17]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    
    # create a table with the dataset
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

We will start a new W&B `run` and put everything into a raw Artifact.

In [18]:
run = wandb.init(
    project=params.WANDB_PROJECT, 
    entity=params.ENTITY, # team name
    job_type="upload"
)
raw_data_at = wandb.Artifact(
    params.RAW_DATA_AT, 
    type="raw_data"
)

In [19]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', ref=None, birth_artifact_id=None, size=1594, extra={}, local_path='/home/mazen/.local/share/wandb/artifacts/staging/tmpqk3xydtu')

Let's add the images and label masks.

In [20]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/home/mazen/.fastai/data/bdd_simple_1k/images)... Done. 0.5s
wandb: Adding directory to artifact (/home/mazen/.fastai/data/bdd_simple_1k/labels)... Done. 0.5s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table. 

In [21]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [22]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our `run`. 

In [23]:
raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='Kuxw4GLE33VTSA9tE8g5ug==', ref=None, birth_artifact_id=None, size=588824, extra={}, local_path='/home/mazen/.local/share/wandb/artifacts/staging/tmpfgl8z5fo')

In [24]:
run.log_artifact(raw_data_at)
run.finish() # upload the information to wandb